In [1]:
import INN
import torch
import torch.nn as nn

In [10]:
model = INN.Sequential(INN.Nonlinear(dim=3, method='RealNVP'),
                       INN.BatchNorm1d(3),
                       INN.Nonlinear(dim=3, method='RealNVP'))
model.eval()
''

''

In [6]:
model = INN.BatchNorm1d(3)
#model.running_var = torch.Tensor([2,2,2])
#model.eval()

In [3]:
def linear_Jacobian_matrix(model, x):
    batch_size, dim = x.shape
    x.requires_grad = True
    model.computing_p(True)
    model.eval()
    y, log_p, log_det = model(x)
    
    grad_list = []
    for i in range(dim):
        v = torch.zeros((batch_size, dim))
        v[:, i] = 1
        grad = INN.utilities.vjp(y, x, v)[0]
        grad_list.append(grad.detach())
    return torch.stack(grad_list, dim=1), log_det

In [4]:
x = torch.randn((4, 3))

In [9]:
Js, log_det = linear_Jacobian_matrix(model, x)
real_log_det = torch.log(torch.abs(torch.det(Js)))

print(f'ground truth={real_log_det},\noutput={log_det}')

ground truth=tensor([-2.3765, -1.4822, -1.9907, -1.3673]),
output=tensor([-2.3765, -1.4822, -1.9907, -1.3673], grad_fn=<AddBackward0>)


## Bug lists

1. `INN.BatchNorm1d()` fails on Jacobian tests
2. `INN.iResNet()` has large differece to the ground-truth!